In [ ]:
### CONNECTION TO DATABASE AND LOADING TABLES ###
import pandas as pd
import warnings
import numpy as np
from sqlalchemy import create_engine
import random
from dotenv import load_dotenv
import os
warnings.filterwarnings("ignore")

db_connection_string = os.environ.get('POSTGRES_CONNECTION')
engine = create_engine(db_connection_string)

user_db = os.environ.get('POSTGRES_USER')
post_db = os.environ.get('POSTGRES_POST')
feed_db = os.environ.get('POSTGRES_FEED')

df_user = pd.read_sql(user_db, engine)
df_post = pd.read_sql(post_db, engine)
df_feed = pd.read_sql(feed_db, engine)


random.seed(42)

In [ ]:
### CREATION OF EMBEDDINGS AND CONCATINATION WITH df_post ###
import tqdm
from transformers import RobertaModel, RobertaTokenizer
import torch

model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaModel.from_pretrained(model_name)

def get_roberta_embeddings(text):
    tokens = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**tokens)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()

embeddings = []
for text in tqdm(df_post['text'], desc="Processing texts", unit=" texts"):
    embedding = get_roberta_embeddings(text)
    embeddings.append(torch.tensor(embedding))

embeddings = torch.cat(embeddings)
df_post['text_embeddings'] = embeddings.tolist()

In [25]:
df_post.head(3)

,post_id,text,topic,pca_tfidf_features,text_embeddings,text_embeddings_mean
0,1,UK economy facing major risks\n\nThe UK manufa...,business,-0.118109,"[0.022685404866933823, 0.1292664110660553, 0.0...",0.022440
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,-0.142662,"[0.017235953360795975, 0.08235780149698257, 0....",0.022096
2,3,Asian quake hits European shares\n\nShares in ...,business,-0.104280,"[-0.03464171290397644, 0.21685758233070374, 0....",0.021477


In [26]:
### USING PCA ON EMBETTINGS AND CONCATINATION WITH df_post ###
from sklearn.decomposition import PCA
pca = PCA(n_components=1)  
pca_embeddings = pca.fit_transform(embeddings)


df_post['pca_embeddings'] = pca_embeddings

In [368]:
df_post.head(3)

,post_id,text,topic,pca_embeddings
0,1,UK economy facing major risks\n\nThe UK manufa...,business,-2.768791
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,-1.971959
2,3,Asian quake hits European shares\n\nShares in ...,business,-1.918611


In [369]:
### OHE FUNCTIONS ###

from sklearn.preprocessing import OneHotEncoder
import pickle

def get_encoder_inst(feature_col):
  
    assert isinstance(feature_col, pd.Series)
    feature_vec = feature_col.sort_values().values.reshape(-1, 1)
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(feature_vec) 
  
    filename = '.pickle'
    pickle.dump(enc, open(filename, 'wb'))
    return enc

def get_one_hot_enc(feature_col, enc,cols):
  
    assert isinstance(feature_col, pd.Series)
    assert isinstance(enc, OneHotEncoder)
    unseen_vec = feature_col.values.reshape(-1, 1)
    encoded_vec = enc.transform(unseen_vec).toarray()
    column_name = enc.get_feature_names([cols])
    encoded_df = pd.DataFrame(encoded_vec, columns= column_name)
    return encoded_df

ohe_cat_list = ['country']
ohe_cat_data = df_user[ohe_cat_list]

df_user.drop(ohe_cat_list,axis=1,inplace=True)

data_list = []
for cols in ohe_cat_data.columns:
    encoder = get_encoder_inst(ohe_cat_data[cols])
    one = get_one_hot_enc(ohe_cat_data[cols],encoder,cols)
    data_list.append(one)
    
final_ohe = pd.concat(data_list,axis=1)
df_user.reset_index(drop=True, inplace=True)
final_ohe.reset_index(drop=True, inplace=True)
for cols in final_ohe.columns:
    final_ohe[cols] = final_ohe[cols].astype('int')


In [370]:
### OHE OF country COLUMN ###

user_ohe = pd.concat([df_user,final_ohe],axis=1)
df_user = user_ohe.copy()

In [371]:
df_user.head(3)

,user_id,gender,age,city,exp_group,os,source,country_Azerbaijan,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,country_Switzerland,country_Turkey,country_Ukraine
0,200,1,34,Degtyarsk,3,Android,ads,0,0,0,0,0,0,0,1,0,0,0
1,201,0,37,Abakan,0,Android,ads,0,0,0,0,0,0,0,1,0,0,0
2,202,1,17,Smolensk,4,Android,ads,0,0,0,0,0,0,0,1,0,0,0


In [372]:
### LABEL ENCODING ###

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_user['os'] = le.fit_transform(df_user['os'])
df_user['source'] = le.fit_transform(df_user['source'])

In [373]:
df_user = df_user.drop('city', axis=1)

In [374]:
df_user.head(3)

,user_id,gender,age,exp_group,os,source,country_Azerbaijan,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,country_Switzerland,country_Turkey,country_Ukraine
0,200,1,34,3,0,0,0,0,0,0,0,0,0,1,0,0,0
1,201,0,37,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,202,1,17,4,0,0,0,0,0,0,0,0,0,1,0,0,0


In [375]:
table_post_all = df_post.copy()

In [376]:
col = 'topic'
one_hot = pd.get_dummies(df_post[col], prefix=col, drop_first=True)
df_post = pd.concat((df_post.drop(col, axis=1), one_hot), axis=1)
df_post = df_post.drop('text', axis=1)

In [377]:
df_feed['day_of_week'] = df_feed['timestamp'].dt.dayofweek
df_feed['hour'] = df_feed['timestamp'].dt.hour

In [378]:
df_feed= df_feed.drop(['timestamp', 'action'], axis=1)

In [379]:
df_feed.head(2)

,user_id,post_id,target,day_of_week,hour
0,74048,6997,0,4,15
1,74048,6085,1,4,15


In [380]:
df = pd.merge(df_feed, df_user, on='user_id', how='left')
df.head(3)

,user_id,post_id,target,day_of_week,hour,gender,age,exp_group,os,source,...,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,country_Switzerland,country_Turkey,country_Ukraine
0,74048,6997,0,4,15,1,43,4,0,0,...,0,0,0,0,0,0,1,0,0,0
1,74048,6085,1,4,15,1,43,4,0,0,...,0,0,0,0,0,0,1,0,0,0
2,74048,6085,0,4,15,1,43,4,0,0,...,0,0,0,0,0,0,1,0,0,0


In [381]:
df.isna().sum()

user_id                0
post_id                0
target                 0
day_of_week            0
hour                   0
gender                 0
age                    0
exp_group              0
os                     0
source                 0
country_Azerbaijan     0
country_Belarus        0
country_Cyprus         0
country_Estonia        0
country_Finland        0
country_Kazakhstan     0
country_Latvia         0
country_Russia         0
country_Switzerland    0
country_Turkey         0
country_Ukraine        0
dtype: int64

In [382]:
df = pd.merge(df, df_post, on='post_id', how='left')
df.head()

,user_id,post_id,target,day_of_week,hour,gender,age,exp_group,os,source,...,country_Switzerland,country_Turkey,country_Ukraine,pca_embeddings,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,74048,6997,0,4,15,1,43,4,0,0,...,0,0,0,0.468564,0,0,1,0,0,0
1,74048,6085,1,4,15,1,43,4,0,0,...,0,0,0,0.448061,0,0,1,0,0,0
2,74048,6085,0,4,15,1,43,4,0,0,...,0,0,0,0.448061,0,0,1,0,0,0
3,74048,4269,0,4,15,1,43,4,0,0,...,0,0,0,-2.310088,0,0,1,0,0,0
4,74048,6155,1,4,15,1,43,4,0,0,...,0,0,0,-1.769268,0,0,1,0,0,0


In [383]:
### SPLIT OF DATAFRAME ###

from sklearn.model_selection import train_test_split

X = df.drop(['user_id', 'post_id', 'target'], axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=1,
                                                    shuffle=True,
                                                    test_size=0.2)

In [384]:
X.columns

Index(['day_of_week', 'hour', 'gender', 'age', 'exp_group', 'os', 'source',
       'country_Azerbaijan', 'country_Belarus', 'country_Cyprus',
       'country_Estonia', 'country_Finland', 'country_Kazakhstan',
       'country_Latvia', 'country_Russia', 'country_Switzerland',
       'country_Turkey', 'country_Ukraine', 'pca_embeddings', 'topic_covid',
       'topic_entertainment', 'topic_movie', 'topic_politics', 'topic_sport',
       'topic_tech'],
      dtype='object')

In [385]:
y_train.sum(), y_test.sum()

(85179, 21379)

In [386]:
X.head(3)

,day_of_week,hour,gender,age,exp_group,os,source,country_Azerbaijan,country_Belarus,country_Cyprus,...,country_Switzerland,country_Turkey,country_Ukraine,pca_embeddings,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,4,15,1,43,4,0,0,0,0,0,...,0,0,0,0.468564,0,0,1,0,0,0
1,4,15,1,43,4,0,0,0,0,0,...,0,0,0,0.448061,0,0,1,0,0,0
2,4,15,1,43,4,0,0,0,0,0,...,0,0,0,0.448061,0,0,1,0,0,0


In [387]:
### MODEL XGBOOST ###

from xgboost import XGBClassifier
from sklearn.metrics import classification_report

model = XGBClassifier()

model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [388]:
predict = model.predict(X_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94    178621
           1       0.00      0.00      0.00     21379

    accuracy                           0.89    200000
   macro avg       0.45      0.50      0.47    200000
weighted avg       0.80      0.89      0.84    200000



In [389]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_test, predict).ravel()

print(f"True Negative Errors: {tn}")
print(f"False Positive Errors: {fp}")
print(f"False Negative Errors: {fn}")
print(f"True Positive Errors: {tp}")

print((classification_report(y_test, predict)))

True Negative Errors: 178619
False Positive Errors: 2
False Negative Errors: 21379
True Positive Errors: 0
              precision    recall  f1-score   support

           0       0.89      1.00      0.94    178621
           1       0.00      0.00      0.00     21379

    accuracy                           0.89    200000
   macro avg       0.45      0.50      0.47    200000
weighted avg       0.80      0.89      0.84    200000



In [308]:
### CHECKER SETTING AND MODEL SAVE ###

import pickle


def get_model_path(path: str) -> str:
    if os.environ.get("IS_LMS") == "1":
        MODEL_PATH = '/workdir/user_input/model'
    else:
        MODEL_PATH = path
    return MODEL_PATH


def load_models():
    model_path = get_model_path("model_dl.pkl")
    model = pickle.load(open(model_path, 'rb'))
    return model


filename = 'model_dl.pkl'
pickle.dump(model, open(filename, 'wb'))
loaded_model = load_models()

In [309]:
filename = 'model_dl.pkl'
pickle.dump(model, open(filename, 'wb'))
loaded_model = load_models()

print(loaded_model.predict_proba(X_test))

[[0.929695   0.07030499]
 [0.94267595 0.05732406]
 [0.8761291  0.12387092]
 ...
 [0.877412   0.12258798]
 [0.91519725 0.08480274]
 [0.9106193  0.08938068]]


In [310]:
df_user_data = df_user.copy()
df_post_text = df_post.copy()

In [311]:
df_post_text

,post_id,pca_embeddings,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,-2.768791,0,0,0,0,0,0
1,2,-1.971959,0,0,0,0,0,0
2,3,-1.918611,0,0,0,0,0,0
3,4,-1.163826,0,0,0,0,0,0
4,5,-0.443954,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
7018,7315,0.105099,0,0,1,0,0,0
7019,7316,0.284611,0,0,1,0,0,0
7020,7317,0.611653,0,0,1,0,0,0
7021,7318,0.368483,0,0,1,0,0,0


In [312]:
### CREATING NEW TABLES IN DATABASE ###

import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(
    os.environ.get('POSTGRES_CONNECTION')
)


df_user_data.to_sql(os.environ.get('POSTGRES_USER_UPLOAD'), con=engine, index=False)
df_post_text.to_sql(os.environ.get('POSTGRES_POST_UPLOAD'), con=engine, index=False)

23

In [313]:
### LOADING BATCHED FEATURES ###

def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        os.environ.get('POSTGRES_CONNECTION')
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)


def load_features() -> pd.DataFrame:
    return batch_load_sql(os.environ.get('POSTGRES_USER_DOWNLOAD'))


def load_features_post() -> pd.DataFrame:
    return batch_load_sql(os.environ.get('POSTGRES_POST_DOWNLOAD'))

In [314]:
load_features_post()

,post_id,pca_embeddings,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,-2.768791,0,0,0,0,0,0
1,2,-1.971959,0,0,0,0,0,0
2,3,-1.918611,0,0,0,0,0,0
3,4,-1.163826,0,0,0,0,0,0
4,5,-0.443954,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
7018,7315,0.105099,0,0,1,0,0,0
7019,7316,0.284611,0,0,1,0,0,0
7020,7317,0.611653,0,0,1,0,0,0
7021,7318,0.368483,0,0,1,0,0,0
